March: Images for PO Seminar presentation - speed slices animations.

In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import cmocean as cmo
import os,sys,fnmatch,time
import numpy as np
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
from matplotlib.ticker import MultipleLocator
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import animation
from salishsea_tools.nc_tools import scDataset
from matplotlib import ticker
from matplotlib import colors
import matplotlib as mpl
from matplotlib.colors import LinearSegmentedColormap
import math
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import matplotlib.patches as patches

import sys
sys.path.append('/ocean/imachuca/Canyons/analysis_mackenzie_canyon/notebooks/general_circulation/')
import general_functions
import quicklook

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [2]:
def get_vars_at_depth_OSM(dirname, fname, maskname, dep_ind):
    
    filesU = general_functions.get_files(dirname, fname, 'grid_U')        
    filesV = general_functions.get_files(dirname, fname, 'grid_V')
    filesT = general_functions.get_files(dirname, fname, 'grid_T')
    print('files')
    
    y,x = slice(1,-1,None), slice(1,-1,None)
    time_ind = 5*24

    with scDataset(filesU) as dsU, scDataset(filesV) as dsV, scDataset(filesT) as dsT:
        vozocrtx0 = dsU.variables['vozocrtx'][:time_ind,dep_ind,y,x]
        print('U')
        vomecrty0 = dsV.variables['vomecrty'][:time_ind,dep_ind,y,x]
        print('V')
        sozotaux = dsU.variables['sozotaux'][:time_ind,0,0]
        depthu = dsU.variables['depthu'][:]
        nav_lat = dsT.variables['nav_lat'][y, x]
        nav_lon = dsT.variables['nav_lon'][y, x]

    with nc.Dataset(os.path.join(dirname, maskname), 'r') as dsM:
        umask0 = dsM.variables['umask'][0,dep_ind,y,x]
        vmask0 = dsM.variables['vmask'][0,dep_ind,y,x]

    umask = np.tile(umask0, (len(sozotaux), 1, 1))
    vmask = np.tile(vmask0, (len(sozotaux), 1, 1))

    vozocrtx = np.ma.array(vozocrtx0, mask=1 - umask)
    vomecrty = np.ma.array(vomecrty0, mask=1 - vmask)
    
    return vozocrtx, vomecrty, umask, vmask, depthu, sozotaux, nav_lat, nav_lon

In [3]:
def get_values(n, vozocrtx, vomecrty, nav_lon, nav_lat, umask):
    U_vel = vozocrtx[n, ...]
    V_vel = vomecrty[n, ...]
    x_slice, y_slice, u_nstg, v_nstg, speeds = quicklook.get_speeds(U_vel, V_vel, 1)
    xs, ys = nav_lon[:-1, :-1], nav_lat[:-1, :-1]
    umask_fixed = umask[0,:-1,:-1]
    return xs, ys, u_nstg, v_nstg, speeds, umask_fixed

In [4]:
wind = 'double'

In [5]:
dep_ind = 11

In [6]:
dirname='/ocean/imachuca/Canyons/results_mackenzie/initial_collection/ideal_' + wind + '/'
fname = "1_MCKNZ_1h_20170101_201701*"
maskname = '1_mesh_mask.nc'

vozocrtx_i, vomecrty_i, umask_i, vmask_i, depthu_i, sozotaux_i, nav_lat_i, nav_lon_i = get_vars_at_depth_OSM(dirname, fname, maskname, dep_ind)

files
U
V


In [7]:
dirname='/ocean/imachuca/Canyons/results_mackenzie/initial_collection/ideal_' + wind + '/'
fname = "MCKNZ_1h_20170101_201701*"
maskname = 'mesh_mask.nc'

vozocrtxp_i, vomecrtyp_i, umaskp_i, vmaskp_i, depthup_i, sozotauxp_i, nav_latp_i, nav_lonp_i = get_vars_at_depth_OSM(dirname, fname, maskname, dep_ind)

files
U
V


In [8]:
dirname='/ocean/imachuca/Canyons/results_mackenzie/initial_collection/real_' + wind + '/'
fname = "1_MCKNZ_1h_20170101_201701*"
maskname = '1_mesh_mask.nc'

vozocrtx_r, vomecrty_r, umask_r, vmask_r, depthu_r, sozotaux_r, nav_lat_r, nav_lon_r = get_vars_at_depth_OSM(dirname, fname, maskname, dep_ind)

FileNotFoundError: [Errno 2] No such file or directory: '/ocean/imachuca/Canyons/results_mackenzie/initial_collection/real_double/'

In [ ]:
dirname='/ocean/imachuca/Canyons/results_mackenzie/initial_collection/real_' + wind + '/'
fname = "MCKNZ_1h_20170101_201701*"
maskname = 'mesh_mask.nc'

vozocrtxp_r, vomecrtyp_r, umaskp_r, vmaskp_r, depthup_r, sozotauxp_r, nav_latp_r, nav_lonp_r = get_vars_at_depth_OSM(dirname, fname, maskname, dep_ind)

In [ ]:
np.arange(0, 0.4, 0.05)

In [ ]:
def plot_streamline_day(fig, ax, vozocrtx, vomecrty, nav_lon, nav_lat, umask,
                        vozocrtxp, vomecrtyp, nav_lonp, nav_latp, umaskp,
                        depthu, dep_ind, n, kind):
    
    ax.cla()
    vmin, vmax = 0, 0.4
    norm = mpl.colors.Normalize(vmin, vmax)
    levels = [0.05, 0.1, 0.2]#np.arange(0, 0.4, 0.05)
    cmap=cmo.cm.matter
    cmap.set_bad('silver')
    
    # child
    xs, ys, u_nstg, v_nstg, speeds, umask_fixed = get_values(n, vozocrtx, vomecrty, nav_lon, nav_lat, umask)
    #parent
    xsp, ysp, u_nstgp, v_nstgp, speedsp, umask_fixedp = get_values(n, vozocrtxp, vomecrtyp, nav_lonp, nav_latp, umaskp)
    
    y_top = 145
    x_top = 50
    x_half= x_top/2
    x_left = x_top + x_half
    x_right = x_top - x_half
    
    if kind == 'ideal':
        # parent        
        pp = ax.pcolormesh(xsp[:y_top, x_top:-x_top], ysp[:y_top, x_top:-x_top], speedsp[:y_top, x_top:-x_top],
                           cmap=cmap, vmin=vmin, vmax=vmax, norm=norm, zorder=1)
        # parent
        csp = ax.contour(xsp[:y_top, x_top:-x_top], ysp[:y_top, x_top:-x_top], speedsp[:y_top, x_top:-x_top],
                         levels = levels, colors='k', alpha=1, zorder=3)
        
    elif kind == 'real':
        # parent
        pp = ax.pcolormesh(xsp[:y_top, x_left:-x_right], ysp[:y_top, x_left:-x_right], speedsp[:y_top, x_left:-x_right],
                           cmap=cmap, vmin=vmin, vmax=vmax, norm=norm, zorder=1)
        # parent
        csp = ax.contour(xsp[:y_top, x_left:-x_right], ysp[:y_top, x_left:-x_right], speedsp[:y_top, x_left:-x_right],
                         levels = levels, colors='k', alpha=1, zorder=3)
    
    # child
    p = ax.pcolormesh(xs, ys, speeds, cmap=cmap, vmin=vmin, vmax=vmax, norm=norm, zorder=2)
    
    if n == 0:
        axins = inset_axes(ax,
                        width="40%",  # width = 10% of parent_bbox width
                        height="5%",  # height : 50%
                        loc=2)
        axins.xaxis.set_ticks_position("bottom")
        cbar = fig.colorbar(p, cax=axins, orientation='horizontal')#, pad=cpad, fraction=cbarf)
        cbar.set_label('speed [ms$^{-1}$]', fontsize=35, rotation=0, labelpad=3.0)
        tick_locator = ticker.MaxNLocator(nbins=4)
        cbar.locator = tick_locator
        cbar.update_ticks()
        cbar.ax.tick_params(labelsize=20)
        
    ax.tick_params(axis='both', which='both', labelsize=20)
    ax.set_title(str(n), fontsize=35)
    ax.set_ylabel('Latitude', fontsize=35)
    ax.set_xlabel('Longitude', fontsize=35)
    ax.clabel(csp, inline=1, fontsize=20, fmt='%1.2f')
    w=1/math.cos(math.radians(70.0))
    ax.set_aspect(w)
    fig.tight_layout()
    
    return

In [ ]:
n = 119
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 15))

plot_streamline_day(fig, ax1, vozocrtx_i, vomecrty_i, nav_lon_i, nav_lat_i, umask_i, 
                    vozocrtxp_i, vomecrtyp_i, nav_lonp_i, nav_latp_i, umaskp_i, depthu_i, dep_ind, n, 'ideal')

plot_streamline_day(fig, ax2, vozocrtx_r, vomecrty_r, nav_lon_r, nav_lat_r, umask_r,
                    vozocrtxp_r, vomecrtyp_r, nav_lonp_r, nav_latp_r, umaskp_r, depthu_r, dep_ind, n, 'real')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (20, 10))

def init():
    return

def animate(n):
    plot_streamline_day(fig, ax1, vozocrtx_i, vomecrty_i, nav_lon_i, nav_lat_i, umask_i, 
                        vozocrtxp_i, vomecrtyp_i, nav_lonp_i, nav_latp_i, umaskp_i, depthu_i, dep_ind, n, 'ideal')

    plot_streamline_day(fig, ax2, vozocrtx_r, vomecrty_r, nav_lon_r, nav_lat_r, umask_r,
                        vozocrtxp_r, vomecrtyp_r, nav_lonp_r, nav_latp_r, umaskp_r, depthu_r, dep_ind, n, 'real')
    
Writer = animation.writers['ffmpeg']
writer = Writer(fps=3, bitrate=1000000, codec="libx264")
anim = animation.FuncAnimation(fig, animate, init_func=init, frames=119, repeat=False)
anim.save('anim_'+wind+'_speeds.mp4', writer=writer)